In [1]:
%%configure -f
{
    "conf": {
           "spark.pyspark.python": "python3",
           "spark.pyspark.virtualenv.enabled": "true",
           "spark.pyspark.virtualenv.type":"native",
           "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
           "spark.jars" : "s3://sagemaker-coherehealth-1/spark/elasticsearch-hadoop-7.10.1.jar,s3://sagemaker-coherehealth-1/spark/org.apache.commons.httpclient.jar"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1624902329262_0016,pyspark,idle,Link,Link,
16,application_1624902329262_0017,pyspark,idle,Link,Link,


In [2]:
%%HTML
<style>
pre {white-space:pre !important;}
</style>

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17,application_1624902329262_0018,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package('boto3==1.17.74')
sc.install_pypi_package('biopython')
sc.install_pypi_package('matplotlib')
sc.install_pypi_package('pandas')
sc.install_pypi_package('ipython')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/63/ec/ce099cd67c8c8a60f2666d97d91e993c2dde968ec6fbaf1d06024591a38f/boto3-1.17.74-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/d0/693477c688348654ddc21dcdce0817653a294aa43f41771084c25e7ff9c7/s3transfer-0.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2f/ae/55b37b5f87c31d391fe2befa0d49c8d7fcad5a2908147ab28c44e661d029/botocore-1.20.102-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5f/64/43575537846896abac0b15c3e5ac678d787a4021e906703f1766bfb8ea11/urllib3-1.26.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/5a/42/de1ed545df624180b84c613e5e4de4848f72989ce5846a74af6baa0737b9/biopython-1.79-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl

  Usin

In [5]:
# df = spark.read.option('header',"true").option("delimeter", "|").csv("s3a://sagemaker-coherehealth-1/HUMANA_MEDCLAIM_201811.txt", sep="|")
df = spark.read.load('s3a://emr-cohere-data-management/humana/med_claims/transformed/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = df.select('member_ssn', 'claim_id', 'service_from_date', 'diagnosis_code1', 'procedure_code')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+-----------------+---------------+--------------+
|member_ssn|       claim_id|service_from_date|diagnosis_code1|procedure_code|
+----------+---------------+-----------------+---------------+--------------+
| 400966111|820211270234971|       2021-05-06|           D181|         11301|
| 249601794|820211270235781|       2021-05-04|          N3000|         A0428|
| 237740988|820211390434029|       2021-04-16|          M5412|         95886|
| 309461548|820211400367502|       2021-04-27|         Z79899|         84460|
| 252319039|820211400331529|       2021-03-25|         Z13220|         84460|
| 291746766|820211400367556|       2021-04-30|         Z20822|         U0003|
| 911161866|820211400359474|       2021-05-12|           N186|         83615|
| 540527529|820211400371164|       2021-05-07|           C021|         38724|
| 141306794|820211330196534|       2021-05-11|        H401133|         92083|
| 991326332|820211321436188|       2021-04-23|           R079|  

In [8]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

604812713

In [9]:
df = df.filter(df.diagnosis_code1.isNotNull())
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

604811764

In [24]:
import boto3 
import pandas as pd
from io import StringIO

path = 'csv/cpt_codes.csv'

s3 = boto3.resource('s3')
codes_csv = s3.Object('sagemaker-coherehealth-1', path).get()['Body'].read().decode('utf-8')

cpt_dict = {couple[0]: couple[1] for couple in [item.split(',') for item in codes_csv.split('\n')]}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
import json

s3 = boto3.resource('s3')
json_str = s3.Object('sagemaker-coherehealth-1', 'claims-data/carepaths.json').get()['Body'].read().decode('utf-8')
data = json.loads(json_str)
data['M545']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'json' is not defined
Traceback (most recent call last):
NameError: name 'json' is not defined



In [32]:
import json

data = json.loads(json_str)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# 99213 99213 99214 99214 99214 99214 72110
# '72110' in data['M545'][0][0]['Diagnostics']
make_readable(' '.join(data['M545'][0][0]['Diagnostics']))
cpt_dict['64493']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'64493'
Traceback (most recent call last):
KeyError: '64493'



In [45]:
# cpt_dict['72110']
data['M545'][0][0].keys()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['Conservative Therapy', 'Diagnostics', 'Non-Surgical Management', 'Advanced Imaging', 'Surgical Management', 'Post-Operative Care'])

In [27]:
def get_label(code):
    try:
        return cpt_dict[code]
    except:
        return code

def make_readable(code_str):
    return ' -> '.join([get_label(code) for code in code_str.split(' ')])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
def test_make_readable():
    test_input = '80053 36415 80061 81003 72100 80061 99213 G0439 72100 99213 85025 83036 84443 G0439 72100 80053 36415 72100 84443 81003 85025 83036 '
    print(make_readable(test_input))
    
test_make_readable()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Comprehen metabolic panel -> Routine venipuncture -> Lipid panel -> "Urinalysis -> X-ray exam of lower spine -> Lipid panel -> "Office/outpatient visit -> G0439 -> X-ray exam of lower spine -> "Office/outpatient visit -> Complete cbc w/auto diff wbc -> Glycated hemoglobin test -> Assay thyroid stim hormone -> G0439 -> X-ray exam of lower spine -> Comprehen metabolic panel -> Routine venipuncture -> X-ray exam of lower spine -> Assay thyroid stim hormone -> "Urinalysis -> Complete cbc w/auto diff wbc -> Glycated hemoglobin test ->

In [10]:
from pyspark.sql import functions as F
import operator

# primary grouping - 
# goal here is to form groups of members and diagnosis codes (i.e. member 1, diag 1 / member 2, diag 1 / member 1 diag 2 ...)
# then for each diag we collect the SET of claim id's and the ordered LIST of procedure codes.
# since spark does not guarantee order in F.collect_list, we form a struct and then sort by service_from_date

# sorter, stolen from StackOverflow
def sorter(l):
    res = sorted(l, key=operator.itemgetter(0))
    ord_ret = ''
    for struct in res:
        ord_ret += str(struct[1]) + ' '
    return ord_ret
    
sort_udf = F.udf(sorter)

grp = (
        df
         .groupby('member_ssn', 'diagnosis_code1')
         .agg(
              F.collect_list(
                             F.struct('service_from_date', 
                                      'procedure_code')
                            ).alias('list_col')
             )
      )
    
list_df = grp.select('member_ssn', 
                     'diagnosis_code1',
                     sort_udf('list_col').alias('ord_procedure_codes'))

list_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+--------------------+
|member_ssn|diagnosis_code1| ord_procedure_codes|
+----------+---------------+--------------------+
|      null|          D4862|99244 77065 99214...|
|      null|           F900|99214 99213 99214...|
|      null|        H4031X0|              99213 |
|      null|         L97818|11043 11046 11045...|
|      null|          M5031|J0702 20610 99213...|
|      null|         M67432|20612 99213 L3808...|
|      null|         O99815|        36415 82947 |
|      null|           P761|74021 71045 74018...|
|      null|         R41841|92523 99214 92523...|
|      null|           R791|71275 93010 93010...|
|      null|        S2001XA|99283 00400 76642...|
|      null|        S32312A|73700 99283 73501...|
|      null|        S3692XA|        99221 99231 |
|      null|        S61219A|99213 12002 90471...|
|      null|        S8251XB|              01480 |
|      null|        S86111D|97110 97161 97010...|
|      null|        S98132D|73630 99203 99213...|


In [11]:
filter_df = list_df.filter(list_df.diagnosis_code1 == 'M545')
filter_df.show(50, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|member_ssn|diagnosis_code1|ord_procedure_codes                                                                                                                                                                                                                                                                             

In [30]:
filter_df.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+--------------------+
|member_ssn|diagnosis_code1| ord_procedure_codes|
+----------+---------------+--------------------+
| 001441451|           M545|99213 99213 99214...|
| 002281800|           M545|        72110 72110 |
| 003347490|           M545|              72148 |
| 008265156|           M545|97161 97140 97112...|
| 009326594|           M545|97110 97162 97140...|
| 009327207|           M545|              72110 |
| 012468031|           M545|              99213 |
| 013177238|           M545|              72100 |
| 015465630|           M545|97140 97161 97140...|
| 022345880|           M545|99213 99213 80307...|
| 023286498|           M545|72100 99203 99204...|
| 023445202|           M545|              99213 |
| 025467017|           M545|97110 97016 97016...|
| 026368496|           M545|        99213 86140 |
| 031362347|           M545|Q0092 R0075 72100...|
| 031383789|           M545|99213 99213 72100...|
| 032380756|           M545|20550 J0702 72100...|


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     CPT_code                         label
0       0003T                 Cervicography
1       0008T   Upper gi endoscopy w/suture
2       0010T     Tb test, gamma interferon
3       0016T  Thermotx choroid vasc lesion
4       0017T  Photocoagulat macular drusen
...       ...                           ...
8217    99511  Home visit, fecal/enema mgmt
8218    99512   Home visit for hemodialysis
8219    99600                Home visit nos
8220    99601    Home infusion/visit, 2 hrs
8221    99602  Home infusion, each addtl hr

[8222 rows x 2 columns]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'CPT_code': 'label', '0003T': 'Cervicography', '0008T': 'Upper gi endoscopy w/suture', '0010T': '"Tb test', '0016T': 'Thermotx choroid vasc lesion', '0017T': 'Photocoagulat macular drusen', '0018T': 'Transcranial magnetic stimul', '0019T': '"Extracorp shock wave tx', '0020T': '"Extracorp shock wave tx', '0021T': '"Fetal oximetry', '0023T': '"Phenotype drug test', '0024T': 'Transcath cardiac reduction', '0026T': 'Measure remnant lipoproteins', '0027T': 'Endoscopic epidural lysis', '0028T': 'Dexa body composition study', '0029T': 'Magnetic tx for incontinence', '0030T': 'Antiprothrombin antibody', '0031T': 'Speculoscopy', '0032T': 'Speculoscopy w/direct sample', '0033T': 'Endovasc taa repr incl subcl', '0034T': 'Endovasc taa repr w/o subcl', '0035T': '"Insert endovasc prosth', '0036T': '"Endovasc prosth', '0037T': 'Artery transpose/endovas taa', '0038T': 'Rad endovasc taa rpr w/cover', '0039T': '"Rad s/i', '0040T': '"Rad s/i', '0041T': 'Detect ur infect agnt w/cpas', '0042T': '"Ct perfu

In [11]:
list_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

80962160

In [12]:
list_df.write.parquet("s3a://sagemaker-coherehealth-1/claims-data/grouped_claims_data/", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# %pip install biopython
from Bio import pairwise2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# # # # this function is iterative! we need a pyspark udf    
# def seq_align_adherence(df):
#     with open('carepaths.json', 'r') as js:
#         data = json.load(js)
    
#     adherence_arr = np.zeros(df.shape[0])
#     for i, row in df.iterrows():
#         if i % 500000 == 0:
#             print(i)
            
#         diag_cd = row['PRIMARY DIAG CD']

#         try:
#             carepaths = data[diag_cd][0]
#             best_adh = 0
#             for cp in carepaths:
#                 target, mapping = get_target_sequence(cp)
#                 pcd_seq, frac = construct_pcd_seq(ast.literal_eval(row['HCPCS CPT4 BASE CD 1']), cp, mapping)
#                 reduced_pcd_seq = get_reduced_str(pcd_seq)
#                 if frac != 0:
#                     adherence = sequence_alignment(reduced_pcd_seq, target) * frac
#                 else:
#                     adherence = 0
#                 if adherence > best_adh:
#                     best_adh = adherence
#             adherence_arr[i] = best_adh
#         except:
#             adherence_arr[i] = -1
            
#     df['sa_adherence'] = adherence_arr

# function to construct scoring dict
def make_scoring_dict(target_str):
    sdict = {}
    for c1 in target_str:
        for c2 in target_str:
            if c1 == c2:
                if c1 != '0' and c2 != '0':
                    sdict[(c1, c2)] = 10
                else:
                    sdict[(c1, c2)] = 1000
            elif c1 == '0' or c2 == '0':
                sdict[(c1, c2)] = -1000
            elif abs(int(c1) - int(c2)) == 1:
                sdict[(c1, c2)] = 5
            else:
                sdict[(c1, c2)] = 0
    return sdict

# function to get target sequence from a list
def get_target_sequence(carepath):
    mapping = {}
    key_list = list(carepath.keys())
    target = '0'
    for k in range(len(key_list)):
        mapping[key_list[k]] = str(k+1)
        target += str(k+1)
        
    return target, mapping


# function to only have unique characters
def get_reduced_str(input_str):
    new = ''
    for c in input_str:
        if c not in new:
            new += c
    return new


# FLAG -- we could roll up on both ends here
# function to query pcd codes and get their service cateogry / construct pcd sequence
def construct_pcd_seq(procedures, carepath, mapping):
    pcd_seq = '0'
    for procedure in procedures:
        for key in carepath:
            if procedure in carepath[key]:
                pcd_seq += mapping[key]
                break
                
    return pcd_seq, (len(pcd_seq)-1) / len(procedures)


def sequence_alignment(pcd_seq, cp_seq):
    scoring = make_scoring_dict(cp_seq)
    a = pairwise2.align.globaldd(cp_seq, 
                                 pcd_seq,
                                 scoring,
                                 -10,
                                 0,
                                 -5,
                                 -1,
                                 penalize_end_gaps=False,
                                 score_only=True)
    
    return ((a-1000) / (len(pcd_seq)-1) / 10)

sequence_alignment('012', '012345')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.0

In [14]:
from pyspark.sql.types import FloatType, DoubleType
import os
import ast
import json

def seq_alignment_adherence(diag_cd, procedure_cds):
    try:
        carepaths = data[diag_cd][0]
        best_adh = 0.0
        for cp in carepaths:
            target, mapping = get_target_sequence(cp)
            code_list = procedure_cds.split(' ')
            pcd_seq, frac = construct_pcd_seq(code_list, cp, mapping)
            reduced_pcd_seq = get_reduced_str(pcd_seq)
            if frac != 0:
                adherence = sequence_alignment(reduced_pcd_seq, target) * frac
                print(adherence)
            else:
                adherence = 0
            if adherence > best_adh:
                best_adh = adherence
        return best_adh
    except:
        return -1.0
    
adherence_udf = F.udf(seq_alignment_adherence, DoubleType())

def generous_adherence(diag_cd, procedure_cds):
    try:
        carepaths = data[diag_cd][0]
        best_adh = 0.0
        for cp in carepaths:
            target, mapping = get_target_sequence(cp)
            code_list = procedure_cds.split(' ')
            pcd_seq, frac = construct_pcd_seq(code_list, cp, mapping)
            reduced_pcd_seq = get_reduced_str(pcd_seq)
            if frac != 0:
                adherence = sequence_alignment(reduced_pcd_seq, target)
            else:
                best_adh = -1.0
            if adherence > best_adh:
                best_adh = adherence
        return best_adh
    except:
        return -1.0
    
gen_adherence_udf = F.udf(generous_adherence, DoubleType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
import boto3
import json

s3 = boto3.resource('s3')
json_str = s3.Object('sagemaker-coherehealth-1', 'claims-data/carepaths.json').get()['Body'].read().decode('utf-8')
data = json.loads(json_str)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
adh_df = list_df.withColumn('seq_align_adh', adherence_udf(list_df.diagnosis_code1, list_df.ord_procedure_codes))
adh_df = adh_df.withColumn('generous_adh', gen_adherence_udf(list_df.diagnosis_code1, list_df.ord_procedure_codes))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
adh_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+--------------------+-------------------+------------+
|member_ssn|diagnosis_code1| ord_procedure_codes|      seq_align_adh|generous_adh|
+----------+---------------+--------------------+-------------------+------------+
|      null|          D4862|99244 77065 99214...|               -1.0|        -1.0|
|      null|           F900|99213 99214 99213...|               -1.0|        -1.0|
|      null|        H4031X0|              99213 |               -1.0|        -1.0|
|      null|         L97818|11043 11046 11042...|               -1.0|        -1.0|
|      null|          M5031|20610 99213 J0702...|0.18617021276595744|         0.5|
|      null|         M67432|L3808 20612 J0702...|               -1.0|        -1.0|
|      null|         O99815|        36415 82947 |               -1.0|        -1.0|
|      null|           P761|74021 71045 74018...|               -1.0|        -1.0|
|      null|         R41841|92523 99214 92507...|               -1.0|        -1.0|
|   

In [18]:
adh_df.write.parquet("s3a://sagemaker-coherehealth-1/claims-data/adherence_data/", mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# ------------------------------

## Start here to read adherence df in most current state

In [5]:
adh_df = spark.read.load('s3a://sagemaker-coherehealth-1/claims-data/adherence_data/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
adh_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

78874665

In [7]:
seq_align_df = adh_df.filter(adh_df.seq_align_adh != -1.0)
seq_align_df.describe(['seq_align_adh']).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|summary|      seq_align_adh|
+-------+-------------------+
|  count|            6855000|
|   mean| 0.1013878628713774|
| stddev|0.21047998251714434|
|    min|                0.0|
|    max| 0.9991680532445923|
+-------+-------------------+

In [8]:
gen_adh_df = adh_df.filter(adh_df.generous_adh != -1.0)
gen_adh_df.describe(['generous_adh']).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|summary|       generous_adh|
+-------+-------------------+
|  count|            2171379|
|   mean| 0.6153987995647674|
| stddev|0.26922224025707536|
|    min|                0.1|
|    max|                1.0|
+-------+-------------------+

In [9]:
import matplotlib
from matplotlib import pyplot as plt 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# fig, ax = plt.subplots()
# ax.hist(list(gen_adh_df.select('generous_adh').toPandas()['generous_adh']), bins=50, color='orange')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# %matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# fig, ax = plt.subplots()
# ax.hist(list(seq_align_df.select('seq_align_adh').toPandas()['seq_align_adh']), bins=50, color='blue')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# %matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
query = (
    gen_adh_df
    .filter(
    (gen_adh_df.generous_adh > 0.5) &
    (gen_adh_df.generous_adh < 0.75) &
    (gen_adh_df.seq_align_adh > 0))
)
query.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+------------+
|member_ssn|diagnosis_code1|ord_procedure_codes                                                                                                                                                                                                                    

In [61]:
rollup_analysis = (
    seq_align_df
    .filter(
        (seq_align_df.seq_align_adh == 0) &
        (seq_align_df.generous_adh != 0) # &
#         (seq_align_df.diagnosis_code1 == 'M545')
    )
)

rollup_analysis.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+-----------------------------------------------------------------------------------------------+-------------+------------+
|member_ssn|diagnosis_code1|ord_procedure_codes                                                                            |seq_align_adh|generous_adh|
+----------+---------------+-----------------------------------------------------------------------------------------------+-------------+------------+
|001507035 |M25571         |73650 73610 J1030 20551 99214 73610 73650 20551 J1030                                          |0.0          |-1.0        |
|004584849 |M25571         |99213                                                                                          |0.0          |-1.0        |
|006403741 |M25511         |86140 73030 85652 99214 77002 83516 20610 20610 77002 86140 89051 None 99213 20610 J1030 73030 |0.0          |-1.0        |
|007349181 |M25562         |A0429 A0425 27520 99213 73562                               

In [16]:
rollup_analysis.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4531642

In [17]:
# could (create list col), explode, analytics???
# an idea - prcd code blacklist? anesthetics, local doctor visits, chiropractor visits?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
sc.install_pypi_package('StringIO')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Could not find a version that satisfies the requirement StringIO (from versions: )
No matching distribution found for StringIO

In [48]:
import pandas as pd
import os
import boto3
from io import StringIO

s3 = boto3.resource('s3')
hierarchy = s3.Object('sagemaker-coherehealth-1', 'claims-data/hierarchy.csv').get()['Body'].read().decode('utf-8')
hdf = pd.read_csv(StringIO(hierarchy))
hdf.at[112, 'End Code'] = 99499
hdf = hdf.astype({'End Code': 'int64'})
hdf[['Start Code', 'End Code']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Start Code  End Code
0           100       222
1           300       352
2           400       474
3           500       580
4           600       670
..          ...       ...
110       99487     99489
111       99495     99496
112       99499     99499
113       99500     99602
114       99605     99607

[115 rows x 2 columns]

In [63]:
hierarchy = hdf.T.to_dict().values()
type(hierarchy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'dict_values'>

In [58]:
def code_to_category(code, hierarchy):
    for h in hierarchy:
        if code >= h['Start Code'] and code <= h['End Code']:
            return h['CPT Minor Category']
    return 'unknown'

code_to_category(99213, hierarchy)
list('ABCEGHJKLMPQRSTV')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['A', 'B', 'C', 'E', 'G', 'H', 'J', 'K', 'L', 'M', 'P', 'Q', 'R', 'S', 'T', 'V']

In [ ]:
hi

In [66]:
import ast
from pyspark.sql.types import FloatType, DoubleType
from pyspark.sql import functions as F

hierarchy = list(hdf.T.to_dict().values())

def construct_rollup_pcd_seq(procedures, carepath, mapping):
    pcd_seq = '0'
    for procedure in procedures:
        if procedure[0] in list('ABCEGHJKLMPQRSTV'):    # if HCPCS
            for key in carepath:
                if procedure in carepath[key]:
                    pcd_seq += mapping[key]
                    break
        else:            # else (CPT-4)
            found = False
            cat = code_to_category(ast.literal_eval(procedure), hierarchy)
            for key in carepath:
                for cp_prcd in carepath[key]:
                    if cat == code_to_category(ast.literal_eval(cp_prcd), hierarchy):
                        pcd_seq += mapping[key]
                        found = True
                        break
                if found:
                    break
            
        
def rollup_adherence(diag_cd, procedure_cds):
    try:
        carepaths = data[diag_cd][0]
        best_adh = 0.0
        for cp in carepaths:
            target, mapping = get_target_sequence(cp)
            code_list = procedure_cds.split(' ')
            pcd_seq, frac = construct_rollup_pcd_seq(code_list, cp, mapping)
            reduced_pcd_seq = get_reduced_str(pcd_seq)
            if frac != 0:
                adherence = sequence_alignment(reduced_pcd_seq, target) * frac
            else:
                adherence = 0
            if adherence > best_adh:
                best_adh = adherence
        return best_adh
    except:
        return -1.0
    
rollup_adherence_udf = F.udf(rollup_adherence, DoubleType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
adh_df = adh_df.withColumn('rollup_adh', rollup_adherence_udf(adh_df.diagnosis_code1, adh_df.ord_procedure_codes))
adh_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+--------------------+-------------+------------+----------+
|member_ssn|diagnosis_code1| ord_procedure_codes|seq_align_adh|generous_adh|rollup_adh|
+----------+---------------+--------------------+-------------+------------+----------+
|      null|        E103313|99213 92134 92273...|         -1.0|        -1.0|      -1.0|
|      null|          E1069|99214 99213 83036...|         -1.0|        -1.0|      -1.0|
|      null|           E319|83520 85598 85613...|         -1.0|        -1.0|      -1.0|
|      null|           F315|99213 90837 90837...|         -1.0|        -1.0|      -1.0|
|      null|         H18892|99283 None 99283 ...|         -1.0|        -1.0|      -1.0|
|      null|         H833X3|99203 99203 97165...|         -1.0|        -1.0|      -1.0|
|      null|           K658|  88305 88307 88307 |         -1.0|        -1.0|      -1.0|
|      null|         L03019|99213 87880 99213...|         -1.0|        -1.0|      -1.0|
|      null|           M724|    